In [68]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

In [69]:
def add_gaussian_noise(features, labels, std=0.1):
    noise = tf.random.normal(shape=tf.shape(features), mean=0.0, stddev=std, dtype=tf.float64)
    noisy_features = features + noise


    noise = tf.random.normal(shape=tf.shape(labels), mean=0.0, stddev=std, dtype=tf.float64)
    noisy_labels = labels + noise

    return noisy_features, noisy_labels

In [70]:

# 1. Load the California housing dataset from scikit-learn
data = fetch_california_housing()
X, y = data.data, data.target

# Reshape y to ensure it is 2D: (num_samples, 1)
y = y.reshape(-1, 1)

# 2. Normalize the features using StandardScaler (optional but recommended)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Split the dataset into training and testing sets
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)



# 4. Create TensorFlow Datasets from the numpy arrays
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))

train_ds = train_ds.map(lambda f, l: add_gaussian_noise(f, l, std=0.03))


test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))

# 5. (Optional) Define a mapping function to ensure the label shapes are consistent
def format_data(features, labels):
    # Ensure labels are 2D tensors (batch_size, 1)
    labels = tf.reshape(labels, (-1, 1))
    return features, labels

# Apply the formatting function
train_ds = train_ds.map(format_data)
test_ds = test_ds.map(format_data)
val_ds = val_ds.map(format_data)

# 6. Batch, shuffle, and prefetch the datasets
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [71]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=50,         # Number of epochs to wait before stopping
    restore_best_weights=True,  # Restore model weights from the epoch with the best val_loss
    mode='min',          # Stop when val_loss stops decreasing
    verbose=1            # Print a message when training stops
)

In [72]:
def custom_metric(y_true, y_pred):

    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    y_true = tf.reshape(y_true, [-1])  # Force 1D shape: (batch_size,)
    y_pred = tf.reshape(y_pred, [-1])  # Force 1D shape: (batch_size,)
    MAE = tf.abs(y_pred - y_true)/y_true
    return tf.reduce_mean(MAE)

#Model1

In [74]:
# 7. Define the Keras model (input shape should match number of features: 8)
model1 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),  # 8 features expected
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Single output for regression
])

# Compile the model with MSE loss and MAE as a metric
model1.compile(optimizer='adam', loss='mse', metrics=['mae'])
model1.summary()

tf.random.set_seed(50)
np.random.seed(50)
random.seed(50)
# 8. Train the model using the training dataset and validate on the test dataset
history = model1.fit(train_ds, validation_data=val_ds, epochs=100 ,callbacks=[early_stopping])

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_45 (Dense)                     │ (None, 64)                  │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_46 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_47 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,689 (10.50 KB)

 Trainable params: 2,689 (10.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 2.5000 - mae: 1.1538 - val_loss: 0.7222 - val_mae: 0.6014
Epoch 2/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.5827 - mae: 0.5485 - val_loss: 0.6006 - val_mae: 0.5071
Epoch 3/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.4722 - mae: 0.4819 - val_loss: 0.6063 - val_mae: 0.4758
Epoch 4/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.4133 - mae: 0.4523 - val_loss: 0.5313 - val_mae: 0.4664
Epoch 5/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.3998 - mae: 0.4425 - val_loss: 0.4729 - val_mae: 0.4580
Epoch 6/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3743 - mae: 0.4322 - val_loss: 0.3806 - val_mae: 0.4464
Epoch 7/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3675 - mae: 0.4282 - val_loss: 0.4946 - val_mae: 0.4463
Epoch 8/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.3683 - mae: 0.4250 - val_loss: 0.4398 - val_mae: 0.4292
Epoch 9/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 10m

In [75]:
model1.compile(optimizer='adam', loss='mse', metrics=['mae'])
loss, mae = model1.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2882 - mae: 0.3627
Test Loss: 0.2947, Test MAE: 0.3613


In [76]:
# 10. (Optional) Make predictions on a batch from the test dataset
for features_batch, labels_batch in test_ds.take(1):
    predictions = model1.predict(features_batch)
    print("Predictions (first 5):", predictions[:5].flatten())
    print("Actual Values (first 5):", labels_batch.numpy()[:5].flatten())

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predictions (first 5): [0.45493472 1.2453809  4.533333   2.6524272  2.6920977 ]
Actual Values (first 5): [0.477   0.458   5.00001 2.186   2.78   ]


#Model 3

In [77]:
# 7. Define the Keras model (input shape should match number of features: 8)
model3 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),  # 8 features expected
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Single output for regression
])

# Compile the model with MSE loss and MAE as a metric
model3.compile(optimizer='adam', loss='mae', metrics=['mae'])
model3.summary()
tf.random.set_seed(50)
np.random.seed(50)
random.seed(50)
# 8. Train the model using the training dataset and validate on the test dataset
history = model3.fit(train_ds, validation_data=val_ds, epochs=100 ,callbacks=[early_stopping])

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_48 (Dense)                     │ (None, 64)                  │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_49 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_50 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,689 (10.50 KB)

 Trainable params: 2,689 (10.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 1.1267 - mae: 1.1267 - val_loss: 0.5513 - val_mae: 0.5513
Epoch 2/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4969 - mae: 0.4969 - val_loss: 0.4795 - val_mae: 0.4795
Epoch 3/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.4500 - mae: 0.4500 - val_loss: 0.4523 - val_mae: 0.4523
Epoch 4/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.4317 - mae: 0.4317 - val_loss: 0.4343 - val_mae: 0.4343
Epoch 5/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.4200 - mae: 0.4200 - val_loss: 0.4256 - val_mae: 0.4256
Epoch 6/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.4153 - mae: 0.4153 - val_loss: 0.4200 - val_mae: 0.4200
Epoch 7/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4127 - mae: 0.4127 - val_loss: 0.4181 - val_mae: 0.4181
Epoch 8/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.4024 - mae: 0.4024 - val_loss: 0.4092 - val_mae: 0.4092
Epoch 9/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms

In [78]:
model3.compile(optimizer='adam', loss='mse', metrics=['mae'])
loss, mae = model3.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2923 - mae: 0.3526
Test Loss: 0.3037, Test MAE: 0.3530


In [79]:
# 10. (Optional) Make predictions on a batch from the test dataset
for features_batch, labels_batch in test_ds.take(1):
    predictions = model3.predict(features_batch)
    print("Predictions (first 5):", predictions[:5].flatten())
    print("Actual Values (first 5):", labels_batch.numpy()[:5].flatten())

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predictions (first 5): [0.42553496 1.1514807  4.6569614  2.4522066  2.7113075 ]
Actual Values (first 5): [0.477   0.458   5.00001 2.186   2.78   ]


#Model2

In [ ]:
X=np.arange(-0.25,0.25,0.0001)
C=400

Fx=((abs(X*C)/(1+abs(X*C)))**40)*abs(X)

Fx=np.where(abs(X)>0.01,abs(X)-0.01,0)

Gx=abs(X)

Hx=X**2

#Fx=(X**4+4*X**2+1)/(X**4+1.76*X**3+3.1*X**2+1.76*X+1)

plt.plot(X,Fx,label='Purposed Loss2', color='blue')
# plt.plot(X,Gx, label='MAE', color='red')
# plt.plot(X,Hx, label='MSE', color='green')

plt.xlabel('X-axis', fontsize=14, fontname='Comic Sans MS')  # Label for the x-axis
plt.ylabel('Y-axis', fontsize=14, fontname='Comic Sans MS')  # Label for the y-axis
#plt.title('High Pass Filter', fontsize=16, fontname='Comic Sans MS')  # Title of the graph
plt.grid(True)
plt.legend()

In [ ]:
X=np.arange(-0.1,0.1,0.0001)
C=400

Fx=((abs(X*C)/(1+abs(X*C)))**40)*abs(X)

Gx=abs(X)

#Fx=(X**4+4*X**2+1)/(X**4+1.76*X**3+3.1*X**2+1.76*X+1)

plt.plot(X,Fx)
plt.plot(X,Gx)

In [ ]:
# Define a custom loss function
def custom_loss(y_true, y_pred):

    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)

    y_true = tf.reshape(y_true, [-1])  # Force 1D shape: (batch_size,)
    y_pred = tf.reshape(y_pred, [-1])  # Force 1D shape: (batch_size,)

    Th=0.05
    conditional_mse = tf.reduce_mean(tf.where(tf.abs(y_true - y_pred)>Th,tf.abs(y_true - y_pred)-Th,0))
    return conditional_mse

In [80]:
def custom_loss(y_true, y_pred):
    C=400
    Poww=40

    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)

    y_true = tf.reshape(y_true, [-1])  # Force 1D shape: (batch_size,)
    y_pred = tf.reshape(y_pred, [-1])  # Force 1D shape: (batch_size,)

    X=tf.abs(y_true-y_pred)

    LOSS=((tf.abs(X*C)/(1+tf.abs(X*C)))**Poww)*tf.abs(X)

    return tf.reduce_mean(LOSS)

In [81]:
# 7. Define the Keras model (input shape should match number of features: 8)
model2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),  # 8 features expected
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Single output for regression
])

# Compile the model with MSE loss and MAE as a metric
model2.compile(optimizer='adam', loss=custom_loss, metrics=['mae'])
model2.summary()

tf.random.set_seed(50)
np.random.seed(50)
random.seed(50)
# 8. Train the model using the training dataset and validate on the test dataset
history = model2.fit(train_ds, validation_data=val_ds, epochs=100,callbacks=[early_stopping])

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_51 (Dense)                     │ (None, 64)                  │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_52 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_53 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,689 (10.50 KB)

 Trainable params: 2,689 (10.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 1.0448 - mae: 1.1325 - val_loss: 0.4686 - val_mae: 0.5482
Epoch 2/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4217 - mae: 0.5001 - val_loss: 0.3957 - val_mae: 0.4726
Epoch 3/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3707 - mae: 0.4474 - val_loss: 0.3754 - val_mae: 0.4524
Epoch 4/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3553 - mae: 0.4313 - val_loss: 0.3593 - val_mae: 0.4357
Epoch 5/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3496 - mae: 0.4255 - val_loss: 0.3503 - val_mae: 0.4263
Epoch 6/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.3384 - mae: 0.4139 - val_loss: 0.3484 - val_mae: 0.4230
Epoch 7/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.3322 - mae: 0.4071 - val_loss: 0.3356 - val_mae: 0.4103
Epoch 8/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3296 - mae: 0.4048 - val_loss: 0.3417 - val_mae: 0.4181
Epoch 9/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/

In [82]:
model2.compile(optimizer='adam', loss=custom_loss, metrics=[custom_metric])
loss, mae = model2.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")
model2.compile(optimizer='adam', loss=custom_loss, metrics=['mae'])
loss, mae = model2.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")
model2.compile(optimizer='adam', loss="mse", metrics=['mae'])
loss, mae = model2.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - custom_metric: 0.1815 - loss: 0.2802
Test Loss: 0.2804, Test MAE: 0.1839
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2802 - mae: 0.3518
Test Loss: 0.2804, Test MAE: 0.3517
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2931 - mae: 0.3518
Test Loss: 0.3030, Test MAE: 0.3517


In [83]:
model2.compile(optimizer='adam', loss=custom_loss, metrics=[custom_metric])
loss, mae = model2.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - custom_metric: 0.1815 - loss: 0.2802
Test Loss: 0.2804, Test MAE: 0.1839


In [17]:
model2.compile(optimizer='adam', loss=custom_loss, metrics=['mae'])
loss, mae = model2.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2792 - mae: 0.3518
Test Loss: 0.2801, Test MAE: 0.3527


In [18]:
model2.compile(optimizer='adam', loss="mse", metrics=['mae'])
loss, mae = model2.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2746 - mae: 0.3518
Test Loss: 0.2812, Test MAE: 0.3527


In [ ]:
# 10. (Optional) Make predictions on a batch from the test dataset
for features_batch, labels_batch in test_ds.take(1):
    predictions = model1.predict(features_batch)
    print("Predictions (first 5):", predictions[:5].flatten())
    print("Actual Values (first 5):", labels_batch.numpy()[:5].flatten())

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predictions (first 5): [0.54721296 1.3883736  4.442125   2.5535917  2.9123209 ]
Actual Values (first 5): [0.477   0.458   5.00001 2.186   2.78   ]


#Statistical Test

In [84]:
from scipy import stats

predictions_1 = model2.predict(test_ds).flatten()
predictions_2 = model3.predict(test_ds).flatten()  # Replace with another model's predictions if available

# 7. Statistical Tests to Compare the Two Sets of Predictions

# Kolmogorov-Smirnov (KS) Test
ks_stat, ks_p_value = stats.ks_2samp(predictions_1, predictions_2)
print("KS Test Statistic:", ks_stat)
print("KS Test p-value:", ks_p_value)

# Independent Samples T-test
t_stat, t_p_value = stats.ttest_ind(predictions_1, predictions_2)
print("T-test Statistic:", t_stat)
print("T-test p-value:", t_p_value)

65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
KS Test Statistic: 0.00872093023255814
KS Test p-value: 0.9975626084699565
T-test Statistic: -0.17022443473591048
T-test p-value: 0.8648378080047401
